In [1]:
'''
This script is used for running IPs through 2 APIs found during a Brute force attack and pulling out the usernames used, then generating an output Excel
file with the results.  

Required Modules that may need to be installed: pandas, extract_msg
Required Files: evidence.txt

Requirements:
1) In the event the log filename is different from what is listed in the next line, 
update the filename in the script or change the filename to match.

2) The logfile(s) needs to be in the same directory as the script unless an absolute path is provided.

No error checking has been included in this file. Its a rushed script

Script Author: larl0ch
'''

import re
import pandas as pd
import urllib.request as urllib3
import json
import time
from datetime import date
import glob
import extract_msg

In [3]:
'''Function for creating evidence from .msg files'''
def gather_evidence():
    result = []
    
    #Path to .MSG
    f = glob.glob(r'SecurityLogs\*.msg')
    
    for filename in f:
        msg = extract_msg.Message(filename)
        result.append(msg.body)

    file_object = open('evidence.txt', 'a')
    for item in result:
        file_object.write(item)
    file_object.close()

    return len(result)

In [5]:
'''Opening the log file, and scaning the data'''
#Location of the log file is in the next line.  Default is 'evidence.txt'
total_logs_searched = gather_evidence()
with open ('evidence.txt') as info:
    lines = info.read().splitlines()

In [6]:
'''Variables used for this script'''
#Regex patterns to find IP addresses and Usernames. NOTE: Does NOT verify valid IP address.
ipmatch = r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b'
#Username pattern needs to be edited to fit the log file's format
usermatch = r'(?<=user )(.*)(?= from)'

#Variables
unique_ips = []
unique_users = []
date = current_date = date.today()

In [7]:
'''Pulls IP addresses from evidence.txt file, removing duplicates, adding to unique_ip list'''
for i in range(len(lines)):
    x = re.findall(usermatch, lines[i]) #Running regex user pattern for matches
    y = re.findall(ipmatch, lines[i])   #Running regex IP pattern for matches
    if y:
        if y[0] not in unique_ips:      #Taking unique matches only
            unique_ips.append(y[0])
    if x:
        if x[0] not in unique_users:    #Taking unique matches only
            unique_users.append(x[0])

In [8]:
'''Function for scanning each IP with https://www.ipqualityscore.com/ and IPWHOIS.APP API'''
def ip_check_scan(unique_ips):
    num = 1
    counts = [11, 22, 33, 44, 55, 66, 77, 88, 99]
    qual_check_results = []
    whois_results = []
    #Enter API Key here
    apiKey = ''
    
    for ip in unique_ips:
        count = len(unique_ips) - num
        print(f"Checking on {ip}... {count} IP's remaining")
        qual_check_response = urllib3.urlopen(f"https://ipqualityscore.com/api/json/ip/{apiKey}/{ip}?strictness=1&lighter_penalties=true")
        whois_response = urllib3.urlopen(f"https://ipwhois.app/json/{ip}?objects=ip,continent,continent_code,country,country_code,isp,org,timezone_name,message")
        qual_check_info = json.load(qual_check_response)
        qual_check_results.append(qual_check_info)
        whois_ip_info = json.load(whois_response)
        whois_results.append(whois_ip_info)
        num += 1
        if num in counts:
            time.sleep(2)
    
    return(qual_check_results, whois_results)  
    

In [ ]:
'''Running ip_check_scan function, and retrieving results'''
#running the checks via API
ip_check_results = ip_check_scan(unique_ips)

#retrieving results
qual_check_results = ip_check_results[0]
whois_results = ip_check_results[1]

In [11]:
'''Generating DataFrames, writer, and xlsx file with separate sheets.'''
#DataFrames
df = pd.DataFrame.from_dict(qual_check_results)
df1 = pd.DataFrame.from_dict(whois_results)
df2 = pd.DataFrame(unique_users)
#df3 = pd.DataFrame(total_logs_searched)

#Writer
output_filename = str(date) + "-evidence_results.xlsx"
writer = pd.ExcelWriter(output_filename)

#Write data to sheets
df.to_excel(writer, "IP Quality Check results", index=False)
df1.to_excel(writer, "IP WHOIS results", index=False)
df2.to_excel(writer, 'Usernames used in attack', index=False)
#df3.to_excel(writer, 'Total logs searched')
#Saving file
writer.save()



In [ ]:
#print(len(total_logs_searched))
print(len(whois_results))
print(len(qual_check_results))